# Advanced Machine Learning 2024/2025 - PyTorch Crash Course

**Teaching Assistant**: Simone Alberto Peirone

Inspired by: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

The goal of this notebook is to train a very basic neural network on the CIFAR 10 dataset.

Then, you can modify the dataset and the training loop to extend the code to a new dataset: Flowers.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import tqdm.auto as tqdm

## Training a network on CIFAR10

### Dataset preparation

In [ ]:
# Transformations to be applied to the output of the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

# Training dataset and dataloader
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)

# Test dataset and dataloader
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)

# TODO: what's the content of images and labels?
# How are multiple images batched together
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))

# print labels
print('Labels: ' + ', '.join(f'{classes[labels[j]].strip()}' for j in range(batch_size)))

## Network definition

Let's define a very basic neural network that takes an image as input and outputs one out of 10 labels.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, n_classes: int = 10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net(n_classes=10)#.cuda()

print(net)

## Optimizers

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training loop

In [ ]:
for epoch in range(2):  # Loop over the dataset twice

    for i, (inputs, labels) in enumerate(trainloader):

        # inputs, labels = inputs, labels

        # inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # Forward the input through the network
        logits = net(inputs)

        # Compute the loss value for the input
        loss = F.cross_entropy(logits, labels)

        # Compute gradients of the loss with respect to the model parameters
        loss.backward()

        # Apply gradient updates
        optimizer.step()

        if i % 1000 == 0:    # print every 2000 mini-batches
            print(f'Training loss: {loss.item():.3f}')


### BONUS: what happens after you call loss.backward()?

Try to inspect the `.grad` property of the network's parameters.

In [ ]:
inputs, labels = next(iter(trainloader))
# inputs, labels = inputs.cuda(), labels.cuda()

# zero the parameter gradients
optimizer.zero_grad()

# Forward the input through the network
logits = net(inputs)

# Compute the loss value for the input
loss = F.cross_entropy(logits, labels)

# Compute the gradients
loss.backward()

net.parameters()

## Validate the network on the test data

In [ ]:
correct = 0
total = 0

with torch.no_grad():

    for inputs, labels in testloader:
        # inputs, labels = inputs.cuda(), labels.cuda()

        # Compute the network logits for the input batch
        outputs = net(inputs) # B x num_classes

        # For each input sample in the batch, we predict the class corresponding to the larget logit
        predicted_labels = outputs.max(1).indices

        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

## Extension to a new dataset

**It's your turn!**.

Download the Flowers dataset from http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz

In [ ]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
!tar -xf 102flowers.tgz && rm 102flowers.tgz

In [ ]:
import os.path as osp
from torch.utils.data import Dataset

from typing import Literal

from torchvision.io import read_image

from PIL import Image

import scipy.io


class FlowersDataset(Dataset):
    def __init__(self, split: Literal['trn', 'val', 'tst'],
                 labels_file: str, splits_path: str, img_dir: str, transform=None):
        self.split = split
        self.ids = np.array(scipy.io.loadmat('setid.mat')[f'{split}id']).squeeze()

        # Load the labels
        labels = np.array(scipy.io.loadmat('imagelabels.mat')['labels'].squeeze())
        self.labels = np.take(labels, self.ids, 0)

        self.img_dir = img_dir

        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_path = osp.join(self.img_dir, f'image_{self.ids[idx]:05d}.jpg')
        image = Image.open(img_path)
        label = self.labels[idx]

        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize(64),
    transforms.CenterCrop(64)
])

trainset = FlowersDataset("trn", "imagelabels.mat", "setid.mat", "jpg", transform=transform)
valset = FlowersDataset("val", "imagelabels.mat", "setid.mat", "jpg", transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
class Net(nn.Module):
    def __init__(self, n_classes: int = 10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 128, 5)
        self.avgpool = nn.AdaptiveAvgPool2d(2)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.avgpool(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


net = Net(n_classes=103)#.cuda()

optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):  # Loop over the dataset twice

    for i, (inputs, labels) in enumerate(trainloader):

        # inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # Forward the input through the network
        logits = net(inputs)

        # Compute the loss value for the input
        loss = F.cross_entropy(logits, labels)

        # Compute gradients of the loss with respect to the model parameters
        loss.backward()

        # Apply gradient updates
        optimizer.step()

        if i % 1000 == 0:    # print every 2000 mini-batches
            print(f'Training loss: {loss.item():.3f}')


In [ ]:
correct = 0
total = 0

with torch.no_grad():

    for inputs, labels in valloader:
        # inputs, labels = inputs.cuda(), labels.cuda()

        # Compute the network logits for the input batch
        outputs = net(inputs)

        # For each input sample in the batch, we predict the class corresponding to the larget logit
        predicted_labels = outputs.max(1).indices

        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total:.2f} %')